In [20]:
from problems.HS100 import HS100, CallableClass
import methods.methods as methods
from typing import Tuple, Union, List, Type
from copy import deepcopy
import pandas as pd
import numpy as np
from numpy import number
from scipy.stats.qmc import LatinHypercube as lhs
from scipy.stats.qmc import scale
import scipy.stats as stats
from smt.surrogate_models import KRG
import random

In [21]:
def sample_sites(problem: dict, n_samples: int, seed=random.randint(1,1000)) -> pd.DataFrame:
    """ Simple wrapper to sample sites
    
    Parameters
    ----------
    problem : dict
        The variables and constraints given

    n_sample: int
        The number of sites to generare

    seed: int
        Seed for the LHS sampling
   
    Returns
    -------
    pd.DataFrame
        The DataFrame of sites
    """

    variables = list(problem["variables"].keys())
    nind = len(variables)
    # Get all the bounds for the variables
    bounds = np.array([problem["variables"][var]["bounds"] for var in variables])
    # Generate the experiment 
    lhs_instance = lhs(
        nind,
        scramble=True,
        strength=1,
        optimization=None,
        seed=seed,
    )
    # Get the experiment
    normalized_array = lhs_instance.random(n_samples)
    # Scale using the bounds
    exp_array = scale(normalized_array, bounds[:, 0], bounds[:, 1])
    # Create the DataFrame
    exp_df = pd.DataFrame(data=exp_array, columns=variables)
    return(exp_df)


In [22]:
def evaluate_sites(local_eval: CallableClass, exp_data: pd.DataFrame, verbose=True):
    """ evaluates sites passed and returns their constraint violation data

        Parameters
        ----------
        local_eval : CallableClass
            The example evaluator to use
        
        n_sample: int
            The number of sites to generare
   
        Returns
        -------
        pd.DataFrame
            The DataFrame of evaluated experimental sites with constraint violation
    """
    eps = 1e-6
    local_problem = local_eval.problem()
    local_eval(exp_data)
    my_constraint_calculator = methods.ConstraintCalculator(local_problem)
    exp_data['__conviol__'] = my_constraint_calculator(exp_data)
    exp_data['__State__'] = pd.cut(exp_data['__conviol__'], [-np.inf, eps, 100*eps, np.inf], include_lowest=True, labels=['Feasible', 'Nearly Feasible', "Infeasible"]).astype("str")
    feasible_sites = (exp_data['__State__'] == 'Feasible').sum()
    percentage = feasible_sites/num_sites * 100
    if verbose: 
        print(f"Test evaluator {local_eval.name} with {len(local_problem['variables'])} variables {percentage}% feasible sites")
    return (exp_data)

In [23]:
def advanced_testing(local_eval: CallableClass, num_sites_training: int, num_sites_testing : int, verbose=True) -> pd.DataFrame:
    """ Wrapper to create an experiment, evaluate the passed in function, create a surrogate model, evaluate model

        Parameters
        ----------
        local_eval : CallableClass
            The example evaluator to use
        
        n_sample_training: int
            The number of sites to train the sm on

        n_sample_testing: int
            The number of sites to test the sm
   
        Returns
        -------
        pd.DataFrame
            The DataFrame of evaluated experimental sites with constraint violation
    """
    # get training data
    sample_data = sample_sites(local_eval.problem(), num_sites_training)
    exp_data = evaluate_sites(local_eval, sample_data, verbose=False)
    variables = list(local_eval.problem()["variables"].keys())
    xt = exp_data[variables].to_numpy()
    yt = exp_data['__conviol__'].to_numpy()

    # create model
    sm = KRG(theta0=[1e-2])
    sm.set_training_values(xt, yt)
    sm.train()

    # get testing data
    eps = 1e-6
    x = sample_sites(local_eval.problem(), num_sites_testing).to_numpy()
    y = sm.predict_values(x).flatten()
    feasible_points = pd.DataFrame(data=x[y < eps], columns=variables)
    
    # evaluate model on testing data
    feasible_points = evaluate_sites(local_eval, feasible_points, verbose=verbose)
    return (feasible_points) 
    

In [24]:
hs100 = HS100()
problem = hs100.problem()
nind = len(problem['variables'])
num_sites = nind**2
test_data = advanced_testing(hs100, num_sites, num_sites**2)
test_data

___________________________________________________________________________
   
                                  Kriging
___________________________________________________________________________
   
 Problem size
   
      # training points.        : 49
   
___________________________________________________________________________
   
 Training
   
   Training ...
   Training - done. Time (sec):  2.2553432
___________________________________________________________________________
   
 Evaluation
   
      # eval points. : 2401
   
   Predicting ...
   Predicting - done. Time (sec):  0.0515921
   
   Prediction time/pt. (sec) :  0.0000215
   
Test evaluator hs100 with 7 variables 20.408163265306122% feasible sites


,x1,x2,x3,x4,x5,x6,x7,f,c1,c2,c3,c4,__conviol__,__State__
0,-6.837194,-4.748830,0.139139,-0.620202,-1.082125,-2.656626,9.670882,1.095573e+04,-1488.456730,343.451329,365.725160,7.489860,1.488457,Infeasible
1,-1.677558,-0.023585,-0.564455,1.031138,-8.545318,-5.127937,1.059622,3.895169e+06,160.409662,281.051110,85.285849,25.519641,0.000000,Feasible
2,-1.166530,0.936686,-2.891258,1.871388,-6.906886,0.621371,7.693130,1.090135e+06,145.386342,194.983676,281.181234,55.200268,0.000000,Feasible
3,-0.306399,-1.273127,2.253022,4.110421,6.608559,-3.026173,2.273288,8.342769e+05,16.052711,239.701251,164.666274,29.158703,0.000000,Feasible
4,-5.991086,1.381304,1.195947,4.738111,1.380442,-0.079316,6.562886,2.813946e+03,-53.604574,302.133120,384.352324,-100.579267,1.007220,Infeasible
5,-1.671787,-0.973026,0.683848,5.162461,9.329821,-2.009676,-2.298670,6.596539e+06,-35.215873,296.112464,190.882170,-23.418469,0.236818,Infeasible
6,-4.651917,-1.328907,-1.793539,-5.034520,-9.194324,-8.618943,6.569811,6.045666e+06,20.742746,282.222524,-91.930499,39.147638,0.919305,Infeasible
7,-7.299864,-0.094002,-2.186769,-2.751005,-7.496682,0.745882,6.196393,1.778089e+06,29.821791,280.815793,410.121151,-156.235307,1.562353,Infeasible
8,-5.016913,-2.609439,-6.042914,6.543540,9.499163,-3.710162,4.294298,7.350222e+06,-275.157892,-37.265773,256.342395,-75.458394,0.885428,Infeasible
9,-3.759119,1.660411,-1.156181,4.851975,6.355516,-1.275653,7.677685,6.633452e+05,-48.852538,291.468603,331.360506,10.153395,0.048853,Infeasible


## Experiment 3
1. Generate training data once and train
2. Regenerate sites until enough pass the model's filter
3. Train again, and repeat

In [25]:
# multi-run training
def experiment_3(local_eval, sites_per_run, runs):
    problem = hs100.problem()
    nind = len(problem['variables'])

    # create model
    sm = KRG(theta0=[1e-2], print_prediction=False)
    
    for i in range(runs):
        if i == 0:
            # get first run's data
            sample_data = sample_sites(local_eval.problem(), sites_per_run)
            exp_data = evaluate_sites(local_eval, sample_data, verbose=False)
        variables = list(local_eval.problem()["variables"].keys())
        xt = exp_data[variables].to_numpy()
        yt = exp_data['__conviol__'].to_numpy()
        
        # train model
        sm.set_training_values(xt, yt)
        sm.train()
    
        # get next run's data
        eps = 1e-6
        x = sample_sites(local_eval.problem(), sites_per_run**2).to_numpy()
        y = sm.predict_values(x).flatten()
        while len(y[y < eps]) < sites_per_run:
            # continue generating sites until enough
            x_p = sample_sites(local_eval.problem(), sites_per_run**2).to_numpy()
            y_p = data=sm.predict_values(x_p).flatten()
            x = np.concatenate((x, x_p))
            y = np.concatenate((y, y_p))
        exp_data = pd.DataFrame(data=x[y < eps], columns=variables)
        
        # evaluate model on testing data
        exp_data = evaluate_sites(local_eval, exp_data)
    return exp_data

In [26]:
hs100 = HS100()
problem = hs100.problem()
nind = len(problem['variables'])
num_sites = 10
#experiment_3(hs100, num_sites, 3)

## Experiment 4
1. Train to maximize -constraint_violation (can do -log, but will be positive infinity sometimes)
2. Filter is now if EI(x) is sufficiently large,
    $EI(x) = (f^*-\mu_f(x))\Phi\left(\frac{f^*-\mu_f(x)}{\sigma_f(x)}\right)+\sigma_f(x)\phi\left(\frac{f^*-\mu_f(x)}{\sigma_f(x)}\right)$
   where $\mu_f,\sigma_f, f^*$ are the predicted value, variance, and maximum of the sm, and $\Phi,\phi$ are normal CDF and PDF.
   
   Note: Need to choose test sites wisely and tune filter better

In [58]:
# weigh with variance
def experiment_4(local_eval, num_sites_training, num_sites_testing,verbose=True):
    # get training data
    sample_data = sample_sites(local_eval.problem(), num_sites_training)
    exp_data = evaluate_sites(local_eval, sample_data, verbose=False)
    variables = list(local_eval.problem()["variables"].keys())
    xt = exp_data[variables].to_numpy()
    yt = (-1)*exp_data['__conviol__'].to_numpy()
    
    # create model
    sm = KRG(theta0=[1e-2], print_global=False)
    sm.set_training_values(xt, yt)
    sm.train()

    # compute EI(x)
    x = sample_sites(local_eval.problem(), num_sites_testing).to_numpy()
    mu = sm.predict_values(x).flatten()
    sigma = np.sqrt(sm.predict_variances(x).flatten())
    y_max = np.max(mu)
    t = (y_max-mu)/sigma
    EI_x = (y_max-mu)*stats.norm.cdf(t) + sigma*stats.norm.pdf(t)
    feasible_points = pd.DataFrame(data=x[EI_x > 5], columns=variables)
    
    # evaluate model on testing data
    feasible_points = evaluate_sites(local_eval, feasible_points, verbose=verbose)
    return (feasible_points) 

In [60]:
hs100 = HS100()
problem = hs100.problem()
nind = len(problem['variables'])
num_sites = 25
experiment_4(hs100, num_sites, num_sites**2)

Test evaluator hs100 with 7 variables 8.0% feasible sites


,x1,x2,x3,x4,x5,x6,x7,f,c1,c2,c3,c4,__conviol__,__State__
0,-2.752108,1.643292,-0.685736,4.380733,-6.357530,-2.346724,7.084188,6.637048e+05,45.685306,280.894280,280.228885,42.154880,0.000000,Feasible
1,-0.530168,0.462062,-6.834167,8.110620,-9.923429,2.667201,1.226627,9.552262e+06,-80.376229,-200.767403,175.109601,-95.327497,2.223949,Infeasible
2,8.837426,8.974352,-0.502793,7.605601,2.276083,4.819393,-5.632602,2.746705e+03,-19731.044559,185.357441,-272.219859,-241.570040,20.063900,Infeasible
3,-2.680765,0.927956,-2.900706,-4.858011,-7.354434,6.366094,2.040145,1.584093e+06,55.674294,211.344122,29.954736,-63.287069,0.632871,Infeasible
4,4.529176,-2.400388,-6.944787,8.647228,9.436024,7.312219,3.649909,7.062605e+06,-352.957638,-224.014890,-205.544882,-213.303148,3.730622,Infeasible
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,-5.122109,8.881293,-6.594075,-4.692807,-1.058255,-0.222830,1.303861,2.917015e+03,-18666.596103,-139.972838,245.064097,-391.801265,19.124640,Infeasible
446,2.358070,7.046119,-7.881060,-5.024913,-3.746630,-6.054276,-9.914397,4.228713e+04,-7353.217625,-375.477679,-207.124166,-225.053257,8.804720,Infeasible
447,3.672497,3.745538,-0.046575,-7.126871,-3.348574,-3.761073,-2.364959,1.561578e+04,-676.797147,248.812514,-6.290188,-33.925072,0.759672,Infeasible
448,-1.821445,-6.059511,-4.959786,7.801219,7.532204,-1.093104,5.227517,1.829286e+06,-4200.334374,66.664844,235.826453,-3.107869,4.200449,Infeasible
